In [13]:
import os, time, json, gc
from datetime import datetime
import numpy as np
import pandas as pd
import joblib
import warnings
warnings.filterwarnings("ignore")

# sklearn / imblearn / boosters
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, average_precision_score
from imblearn.over_sampling import SMOTE
# Boosters
import xgboost as xgb
import lightgbm as lgb

# --------------------
# USER CONFIG (edit if needed)
# --------------------
DATA_PATH = "/content/drive/MyDrive/AIML Dataset.csv"  # existing file path (your original CSV)
CHECKPOINT_DIR = "/content/drive/MyDrive/fraud_detection_checkpoints/epoch_checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
RANDOM_STATE = 42

# Sampling targets
NONFRAUD_SAMPLE_N = 200_000   # sample non-fraud to this many
FRAUD_TARGET_N = 200_000      # expand fraud to this many via SMOTE

# Train/test split
TEST_SIZE = 0.30  # 70/30 train/test

# Epoch/training settings
EPOCHS = 50
# For RandomForest: number of trees added each epoch (epochs * trees_per_epoch = final trees)
RF_TREES_PER_EPOCH = 6   # 6 * 50 = 300 trees (matches your previous RF)
# For boosters: number of boosting rounds added per epoch
XGB_ROUNDS_PER_EPOCH = 6   # 6 * 50 = 300 rounds
LGB_ROUNDS_PER_EPOCH = 6

# Which models to run (any subset of ['rf','xgb','lgb'])
MODELS_TO_RUN = ['rf','xgb','lgb']

# File to persist progress (so we can resume)
PROGRESS_JSON = os.path.join(CHECKPOINT_DIR, "training_progress.json")

In [14]:
# --------------------
# Helpers
# --------------------
def save_progress(state: dict):
    with open(PROGRESS_JSON, "w") as f:
        json.dump(state, f)

def load_progress():
    if os.path.exists(PROGRESS_JSON):
        with open(PROGRESS_JSON, "r") as f:
            return json.load(f)
    return {}

def print_metrics(y_true, y_pred, y_proba=None):
    print(classification_report(y_true, y_pred, digits=4))
    if y_proba is not None and len(np.unique(y_true))>1:
        print("ROC-AUC:", round(roc_auc_score(y_true, y_proba), 6))
        print("PR-AUC:", round(average_precision_score(y_true, y_proba), 6))


In [15]:
# --------------------
# 0) Load data & feature engineering
# --------------------
assert os.path.exists(DATA_PATH), f"Data not found at {DATA_PATH}"
print("Loading raw data...")
df = pd.read_csv(DATA_PATH)
print("Rows, cols:", df.shape)

# Feature engineering (kept exactly as you requested)
print("Creating engineered features...")
df['orig_balance_change'] = df['oldbalanceOrg'] - df['newbalanceOrig']
df['dest_balance_change'] = df['newbalanceDest'] - df['oldbalanceDest']
df['orig_balance_zero'] = (df['oldbalanceOrg'] == 0).astype(int)
df['dest_balance_zero'] = (df['oldbalanceDest'] == 0).astype(int)
df['orig_rel_change'] = df['orig_balance_change'] / (df['oldbalanceOrg'] + 1e-6)
df['dest_rel_change'] = df['dest_balance_change'] / (df['oldbalanceDest'] + 1e-6)
df['zero_transfer'] = ((df['amount'] > 0) & (df['newbalanceDest'] == 0)).astype(int)
df['same_account'] = (df['nameOrig'] == df['nameDest']).astype(int)
df['log_amount'] = np.log1p(df['amount'])

# Columns used by your pipeline earlier
feature_cols = [
    'oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest',
    'isFlaggedFraud','orig_balance_change','dest_balance_change',
    'orig_balance_zero','dest_balance_zero','orig_rel_change','dest_rel_change',
    'zero_transfer','same_account','log_amount','type'
]

# Drop unused cols for modelling stage (keep original df intact)
df_model = df.copy()
# ensure target exists and is correct dtype
df_model['isFraud'] = df_model['isFraud'].astype(int)


Loading raw data...
Rows, cols: (6362620, 11)
Creating engineered features...


In [16]:
#--------------------
# 1) Construct balanced dataset: sample non-fraud 200k and expand fraud via SMOTE to 200k
# --------------------
print("Preparing balanced dataset using sampling + SMOTE...")

# Separate classes
df_nonfraud = df_model[df_model['isFraud']==0]
df_fraud = df_model[df_model['isFraud']==1]

print("Original counts:", df_model['isFraud'].value_counts().to_dict())
print("Non-fraud total available:", len(df_nonfraud))
print("Fraud total available:", len(df_fraud))

# Sample non-fraud
if len(df_nonfraud) < NONFRAUD_SAMPLE_N:
    raise ValueError(f"Not enough non-fraud rows to sample {NONFRAUD_SAMPLE_N}")
nonfraud_sample = df_nonfraud.sample(n=NONFRAUD_SAMPLE_N, random_state=RANDOM_STATE).reset_index(drop=True)
fraud_df = df_fraud.reset_index(drop=True)  # all fraud rows (8213 expected)

print("Sampled non-fraud:", len(nonfraud_sample), "fraud rows:", len(fraud_df))

# Combine for preprocessor fitting & SMOTE
temp = pd.concat([nonfraud_sample, fraud_df], axis=0).reset_index(drop=True)
X_temp = temp[feature_cols].copy()
y_temp = temp['isFraud'].copy()

# Build preprocessor (same as training pipeline)
num_cols = [c for c in feature_cols if c != 'type']
cat_cols = ['type']

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
], remainder='drop')

print("Fitting preprocessor on combined sample (non-fraud sample + fraud rows)...")
preprocessor.fit(X_temp)

print("Transforming temp data to numeric matrix for SMOTE...")
X_temp_enc = preprocessor.transform(X_temp)

# Now apply SMOTE to expand minority class to FRAUD_TARGET_N
print(f"Running SMOTE to make fraud class = {FRAUD_TARGET_N}...")
sm = SMOTE(sampling_strategy={1: FRAUD_TARGET_N}, random_state=RANDOM_STATE)
X_res, y_res = sm.fit_resample(X_temp_enc, y_temp)

print("After SMOTE sizes: total:", X_res.shape[0], "fraud count:", int((y_res==1).sum()), "non-fraud count:", int((y_res==0).sum()))

# Reconstruct DataFrame of encoded features for modeling (we only need arrays for model training)
# We'll keep X_res (numpy) and y_res

Preparing balanced dataset using sampling + SMOTE...
Original counts: {0: 6354407, 1: 8213}
Non-fraud total available: 6354407
Fraud total available: 8213
Sampled non-fraud: 200000 fraud rows: 8213
Fitting preprocessor on combined sample (non-fraud sample + fraud rows)...
Transforming temp data to numeric matrix for SMOTE...
Running SMOTE to make fraud class = 200000...
After SMOTE sizes: total: 400000 fraud count: 200000 non-fraud count: 200000


In [17]:
# --------------------
# 2) Train/test split (70/30)
# --------------------
print("Splitting into train/test (70/30)...")
X_train_enc, X_test_enc, y_train, y_test = train_test_split(
    X_res, y_res, test_size=TEST_SIZE, stratify=y_res, random_state=RANDOM_STATE
)

print("Train shapes:", X_train_enc.shape, y_train.shape, "Test shapes:", X_test_enc.shape, y_test.shape)

# free memory
del X_temp, y_temp, X_temp_enc, X_res, y_res, temp, nonfraud_sample, df_nonfraud, df_fraud
gc.collect()


Splitting into train/test (70/30)...
Train shapes: (280000, 18) (280000,) Test shapes: (120000, 18) (120000,)


10592

In [18]:
# --------------------
# 3) Setup incremental models & resume logic
# --------------------
progress = load_progress()
start_epoch = progress.get('last_epoch', 0)
model_progress = progress.get('models', {})  # track per-model epoch completed

# Prepare objects to hold current models/boosters
rf_model = None
xgb_booster = None
lgb_booster = None

# If prior checkpoints exist for a model, load latest per model
def latest_checkpoint_for(model_name):
    files = sorted([f for f in os.listdir(CHECKPOINT_DIR) if f.startswith(model_name+"_epoch_")])
    return os.path.join(CHECKPOINT_DIR, files[-1]) if files else None

# If resuming, try to load last checkpoint per model
if start_epoch > 0:
    if 'rf' in MODELS_TO_RUN:
        ck = latest_checkpoint_for("rf")
        if ck:
            print("Loading RF checkpoint:", ck)
            rf_model = joblib.load(ck)
            model_progress['rf'] = int(ck.split("_epoch_")[1].split(".pkl")[0])
    if 'xgb' in MODELS_TO_RUN:
        ck = latest_checkpoint_for("xgb")
        if ck:
            print("Loading XGB checkpoint:", ck)
            xgb_booster = xgb.Booster()
            xgb_booster.load_model(ck)
            model_progress['xgb'] = int(ck.split("_epoch_")[1].split(".model")[0])
    if 'lgb' in MODELS_TO_RUN:
        ck = latest_checkpoint_for("lgb")
        if ck:
            print("Loading LGB checkpoint:", ck)
            lgb_booster = lgb.Booster(model_file=ck)
            model_progress['lgb'] = int(ck.split("_epoch_")[1].split(".txt")[0])


Loading RF checkpoint: /content/drive/MyDrive/fraud_detection_checkpoints/epoch_checkpoints/rf_epoch_1.pkl
Loading XGB checkpoint: /content/drive/MyDrive/fraud_detection_checkpoints/epoch_checkpoints/xgb_epoch_1.model


In [20]:
# === FIXED TRAINING LOOP (resumable, stable, no verbosity errors, correct vars) ===
import datetime, gc, os, time, joblib
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
import xgboost as xgb
import lightgbm as lgb

print("\n🚀 Starting training loop (resumable)...\n")

# ✅ Use encoded variables from previous cell
X_train_t = X_train_enc
X_test_t  = X_test_enc

# === Initialize models ===
rf = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)
xgb_model = None
lgb_booster = None

# === Define XGBoost and LightGBM parameters ===
xgb_params = {
    "objective": "binary:logistic",
    "eval_metric": "aucpr",
    "tree_method": "hist",
    "learning_rate": 0.1,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "random_state": RANDOM_STATE
}

lgb_params = {
    "objective": "binary",
    "metric": ["auc", "average_precision"],
    "learning_rate": 0.1,
    "num_leaves": 31,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "seed": RANDOM_STATE
}

for epoch in range(1, EPOCHS + 1):
    print(f"\n=== EPOCH {epoch}/{EPOCHS} ===  {datetime.datetime.now().isoformat()}")
    try:
        # === RANDOM FOREST ===
        rf.set_params(n_estimators=epoch * RF_TREES_PER_EPOCH)
        rf.fit(X_train_t, y_train)
        y_pred = rf.predict(X_test_t)
        y_proba = rf.predict_proba(X_test_t)[:, 1]

        print(f"\n[RF] Classification report (threshold=0.5):")
        print(classification_report(y_test, y_pred, digits=4))
        print("ROC-AUC:", round(roc_auc_score(y_test, y_proba), 6))
        print("PR-AUC:", round(average_precision_score(y_test, y_proba), 6))

        rf_path = os.path.join(CHECKPOINT_DIR, f"rf_epoch_{epoch}.pkl")
        joblib.dump(rf, rf_path)
        print(f"[RF] ✅ Saved checkpoint: {rf_path}")

        # === XGBOOST ===
        dtrain = xgb.DMatrix(X_train_t, label=y_train)
        dtest = xgb.DMatrix(X_test_t, label=y_test)

        print(f"[XGB] Training +{XGB_ROUNDS_PER_EPOCH} rounds (resume={xgb_model is not None})")
        xgb_model = xgb.train(
            xgb_params,
            dtrain,
            num_boost_round=epoch * XGB_ROUNDS_PER_EPOCH,
            xgb_model=xgb_model
        )
        preds = xgb_model.predict(dtest)
        y_pred = (preds >= 0.5).astype(int)

        print(f"[XGB] Classification report (threshold=0.5):")
        print(classification_report(y_test, y_pred, digits=4))
        print("ROC-AUC:", round(roc_auc_score(y_test, preds), 6))
        print("PR-AUC:", round(average_precision_score(y_test, preds), 6))

        xgb_model.save_model(os.path.join(CHECKPOINT_DIR, f"xgb_epoch_{epoch}.model"))
        print(f"[XGB] ✅ Saved checkpoint: xgb_epoch_{epoch}.model")

        # === LIGHTGBM ===
        lgb_train = lgb.Dataset(X_train_t, label=y_train)
        lgb_eval = lgb.Dataset(X_test_t, label=y_test, reference=lgb_train)

        print(f"[LGB] Training +{LGB_ROUNDS_PER_EPOCH} rounds (resume={lgb_booster is not None})")
        lgb_booster = lgb.train(
            lgb_params,
            lgb_train,
            valid_sets=[lgb_train, lgb_eval],
            num_boost_round=epoch * LGB_ROUNDS_PER_EPOCH,
            init_model=lgb_booster,
            callbacks=[
                lgb.early_stopping(stopping_rounds=20),
                lgb.log_evaluation(period=50)
            ]
        )
        preds = lgb_booster.predict(X_test_t)
        y_pred = (preds >= 0.5).astype(int)

        print(f"[LGB] Classification report (threshold=0.5):")
        print(classification_report(y_test, y_pred, digits=4))
        print("ROC-AUC:", round(roc_auc_score(y_test, preds), 6))
        print("PR-AUC:", round(average_precision_score(y_test, preds), 6))

        lgb_path = os.path.join(CHECKPOINT_DIR, f"lgb_epoch_{epoch}.txt")
        lgb_booster.save_model(lgb_path)
        print(f"[LGB] ✅ Saved checkpoint: {lgb_path}")

    except Exception as e:
        print(f"⚠️ Error in epoch {epoch}: {e}")
        continue

    gc.collect()
    time.sleep(1)

print("\n✅ Training Complete. Checkpoints saved for all epochs.")



🚀 Starting training loop (resumable)...


=== EPOCH 1/50 ===  2025-10-12T10:09:19.495498

[RF] Classification report (threshold=0.5):
              precision    recall  f1-score   support

           0     0.9985    0.9977    0.9981     60000
           1     0.9977    0.9985    0.9981     60000

    accuracy                         0.9981    120000
   macro avg     0.9981    0.9981    0.9981    120000
weighted avg     0.9981    0.9981    0.9981    120000

ROC-AUC: 0.999324
PR-AUC: 0.998952
[RF] ✅ Saved checkpoint: /content/drive/MyDrive/fraud_detection_checkpoints/epoch_checkpoints/rf_epoch_1.pkl
[XGB] Training +6 rounds (resume=False)
[XGB] Classification report (threshold=0.5):
              precision    recall  f1-score   support

           0     0.9972    0.9817    0.9894     60000
           1     0.9819    0.9972    0.9895     60000

    accuracy                         0.9894    120000
   macro avg     0.9896    0.9894    0.9894    120000
weighted avg     0.9896    0.9894    